In [1]:
from __future__ import print_function

from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot,merge, concatenate
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from functools import reduce
import tarfile
import numpy as np
import re

Using TensorFlow backend.


In [2]:
def tokenize(sent):
    # splitting the sentences based on spaces and tokenizing them
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]



In [3]:
def parse_stories(lines, only_supporting=False):
   
    # Trying to retrive the questions and 
    # ansers to the particular questions
    
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
        
    return data


In [4]:
def get_stories(f, only_supporting=False, max_length=None):
    
    # Discaring the stories longer than the max length defined 
    # and converting the lines in to a single story corresponding 
    # to the questions
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data


In [5]:
def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):

    # Mapping the length of stories against story with maximum
    # length and doing necessary padding
    
    X = []
    Xq = []
    Y = []
    for story, query, answer in data:
        x=[word_idx[w] for w in story]
        xq=[word_idx[w] for w in query]
        y=np.zeros(len(word_idx) + 1)
        y[word_idx[answer]]=1
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    return (pad_sequences(X, maxlen=story_maxlen),
            pad_sequences(Xq, maxlen=query_maxlen),
            np.array(Y))


In [10]:
''' Accessing the particula modules of the 
Babi dataset saperated by the modules
'''

try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)


In [12]:
challenges = {'simple-negation': 'tasks_1-20_v1-2/en-10k/qa9_simple-negation_{}.txt'}
challenge_type = 'simple-negation'
challenge = challenges[challenge_type]

print('Extracting stories for the challenge:', challenge_type)
train_stories = get_stories(tar.extractfile(challenge.format('train')))
test_stories = get_stories(tar.extractfile(challenge.format('test')))

Extracting stories for the challenge: simple-negation


C:\Users\pruth\Anaconda3\lib\re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [13]:
print(train_stories)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [9]:
print(test_stories)

[(['John', 'travelled', 'to', 'the', 'hallway', '.', 'Mary', 'journeyed', 'to', 'the', 'bathroom', '.'], ['Where', 'is', 'John', '?'], 'hallway'), (['John', 'travelled', 'to', 'the', 'hallway', '.', 'Mary', 'journeyed', 'to', 'the', 'bathroom', '.', 'Daniel', 'went', 'back', 'to', 'the', 'bathroom', '.', 'John', 'moved', 'to', 'the', 'bedroom', '.'], ['Where', 'is', 'Mary', '?'], 'bathroom'), (['John', 'travelled', 'to', 'the', 'hallway', '.', 'Mary', 'journeyed', 'to', 'the', 'bathroom', '.', 'Daniel', 'went', 'back', 'to', 'the', 'bathroom', '.', 'John', 'moved', 'to', 'the', 'bedroom', '.', 'John', 'went', 'to', 'the', 'hallway', '.', 'Sandra', 'journeyed', 'to', 'the', 'kitchen', '.'], ['Where', 'is', 'Sandra', '?'], 'kitchen'), (['John', 'travelled', 'to', 'the', 'hallway', '.', 'Mary', 'journeyed', 'to', 'the', 'bathroom', '.', 'Daniel', 'went', 'back', 'to', 'the', 'bathroom', '.', 'John', 'moved', 'to', 'the', 'bedroom', '.', 'John', 'went', 'to', 'the', 'hallway', '.', 'Sandra

In [14]:
vocab = set()
for story, q, answer in train_stories + test_stories:
    print(story)
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [15]:
with open('story_maxlen_simple_negation.txt','w') as file:
    file.write(str(story_maxlen))
    file.write('\n')
file.close()


with open('query_maxlen_simple_negation.txt','w') as file:
    file.write(str(query_maxlen))
    file.write('\n')
file.close()


In [16]:
''' Knowing some facts and stats about the 
module of dataset we are using
'''

print('-')
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('-')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])
print('-')
print('Vectorizing the word sequences...')

word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
idx_word = dict((i+1, c) for i,c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize_stories(train_stories,
                                                               word_idx,
                                                               story_maxlen,
                                                               query_maxlen)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories,
                                                            word_idx,
                                                            story_maxlen,
                                                            query_maxlen)

-
Vocab size: 27 unique words
Story max length: 74 words
Query max length: 6 words
Number of training stories: 10000
Number of test stories: 1000
-
Here's what a "story" tuple looks like (input, query, answer):
(['Mary', 'is', 'no', 'longer', 'in', 'the', 'bedroom', '.', 'Daniel', 'moved', 'to', 'the', 'hallway', '.'], ['Is', 'Mary', 'in', 'the', 'bedroom', '?'], 'no')
-
Vectorizing the word sequences...


In [17]:
with open('word_idx_simple_negation.txt','w') as file:
    file.write(str(word_idx))
file.close()

In [18]:
with open('idx_word_simple_negation.txt','w') as file:
    file.write(str(idx_word))
file.close()

In [19]:
print(word_idx)

{'went': 25, '?': 2, 'Is': 4, 'to': 23, 'in': 13, '.': 1, 'kitchen': 16, 'longer': 17, 'hallway': 12, 'garden': 11, 'Sandra': 7, 'bathroom': 9, 'the': 22, 'office': 21, 'John': 5, 'Mary': 6, 'no': 19, 'not': 20, 'back': 8, 'journeyed': 15, 'is': 14, 'bedroom': 10, 'Daniel': 3, 'travelled': 24, 'yes': 26, 'moved': 18}


In [20]:
print(idx_word)

{1: '.', 2: '?', 3: 'Daniel', 4: 'Is', 5: 'John', 6: 'Mary', 7: 'Sandra', 8: 'back', 9: 'bathroom', 10: 'bedroom', 11: 'garden', 12: 'hallway', 13: 'in', 14: 'is', 15: 'journeyed', 16: 'kitchen', 17: 'longer', 18: 'moved', 19: 'no', 20: 'not', 21: 'office', 22: 'the', 23: 'to', 24: 'travelled', 25: 'went', 26: 'yes'}


In [21]:
print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-')
print('Compiling...')

-
inputs: integer tensor of shape (samples, max_length)
inputs_train shape: (10000, 74)
inputs_test shape: (1000, 74)
-
queries: integer tensor of shape (samples, max_length)
queries_train shape: (10000, 6)
queries_test shape: (1000, 6)
-
answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_train shape: (10000, 27)
answers_test shape: (1000, 27)
-
Compiling...


In [22]:
train_epochs = 120
batch_size = 32
lstm_size = 64

In [23]:
# initiating 
input_sequence = Input((story_maxlen,))
question = Input((query_maxlen,))

print('Input sequence:', input_sequence)
print('Question:', question)

# defining the encoder for the input and embedding the vocab
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=64))
input_encoder_m.add(Dropout(0.3))

# mapping the dimensions according to the maximum length on input
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=query_maxlen))
input_encoder_c.add(Dropout(0.3))

# vectorizing the questions
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=query_maxlen))
question_encoder.add(Dropout(0.3))


# conctinating input stories and questions
input_encoded_m = input_encoder_m(input_sequence)
print('Input encoded m', input_encoded_m)
input_encoded_c = input_encoder_c(input_sequence)
print('Input encoded c', input_encoded_c)
question_encoded = question_encoder(question)
print('Question encoded', question_encoded)


# cross refrencing the questions against the stories
match = merge([input_encoded_m, question_encoded], mode='dot', dot_axes=(2, 2))
print(match.shape)
match = Activation('softmax')(match)
print('Match shape', match)

response = merge([match, input_encoded_c], mode='sum')
response = Permute((2, 1))(response)  
print('Response shape', response)

# mapping answers against the vector of question and stories
answer = merge([response, question_encoded], mode='concat')
print('Answer shape', answer)

answer = LSTM(64)(answer)  
answer = Dropout(0.3)(answer)
answer = Dense(vocab_size)(answer)  

# popping out prob. of answers
answer = Activation('softmax')(answer)

Input sequence: Tensor("input_1:0", shape=(?, 74), dtype=float32)
Question: Tensor("input_2:0", shape=(?, 6), dtype=float32)
Input encoded m Tensor("sequential_1/dropout_1/cond/Merge:0", shape=(?, 74, 64), dtype=float32)
Input encoded c Tensor("sequential_2/dropout_2/cond/Merge:0", shape=(?, 74, 6), dtype=float32)
Question encoded Tensor("sequential_3/dropout_3/cond/Merge:0", shape=(?, 6, 64), dtype=float32)
(?, 74, 6)
Match shape Tensor("activation_1/truediv:0", shape=(?, 74, 6), dtype=float32)
Response shape Tensor("permute_1/transpose:0", shape=(?, 6, 74), dtype=float32)


C:\Users\pruth\Anaconda3\lib\site-packages\ipykernel\__main__.py:38: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
C:\Users\pruth\Anaconda3\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
C:\Users\pruth\Anaconda3\lib\site-packages\ipykernel\__main__.py:43: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
C:\Users\pruth\Anaconda3\lib\site-packages\ipykernel\__main__.py:48: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Answer shape Tensor("merge_3/concat:0", shape=(?, 6, 138), dtype=float32)


In [24]:
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
model.fit([inputs_train, queries_train], answers_train, batch_size, train_epochs,
          validation_data=([inputs_test, queries_test], answers_test))

Train on 10000 samples, validate on 1000 samples
Epoch 1/120
10000/10000 [==============================] - 8s - loss: 0.7410 - acc: 0.5985 - val_loss: 0.6499 - val_acc: 0.6380
Epoch 2/120
10000/10000 [==============================] - 7s - loss: 0.6473 - acc: 0.6207 - val_loss: 0.6375 - val_acc: 0.6260
Epoch 3/120
10000/10000 [==============================] - 6s - loss: 0.6381 - acc: 0.6242 - val_loss: 0.6384 - val_acc: 0.6200
Epoch 4/120
10000/10000 [==============================] - 6s - loss: 0.6320 - acc: 0.6294 - val_loss: 0.6320 - val_acc: 0.6290
Epoch 5/120
10000/10000 [==============================] - 6s - loss: 0.6286 - acc: 0.6316 - val_loss: 0.6313 - val_acc: 0.6280
Epoch 6/120
10000/10000 [==============================] - 6s - loss: 0.6274 - acc: 0.6302 - val_loss: 0.6316 - val_acc: 0.6270
Epoch 7/120
10000/10000 [==============================] - 6s - loss: 0.6251 - acc: 0.6365 - val_loss: 0.6319 - val_acc: 0.6440
Epoch 8/120
10000/10000 [==============================

In [ ]:
''' Getting the feel for the test dataset
and how it is devided.
'''

In [26]:
for i in range(0,10):
        current_inp = test_stories[i]
        current_story, current_query, current_answer = vectorize_stories([current_inp], word_idx, story_maxlen, query_maxlen)
        current_prediction = model.predict([current_story, current_query])
        current_prediction = idx_word[np.argmax(current_prediction)]
        print(' '.join(current_inp[0]), ' '.join(current_inp[1]), '| Prediction:', current_prediction, '| Ground Truth:', current_inp[2])
        print("-----------------------------------------------------------------------------------------")
        

John is in the hallway . Sandra is in the kitchen . Is Sandra in the bedroom ? | Prediction: no | Ground Truth: no
-----------------------------------------------------------------------------------------
John is in the hallway . Sandra is in the kitchen . Sandra journeyed to the bedroom . Mary travelled to the garden . Is Sandra in the bedroom ? | Prediction: yes | Ground Truth: yes
-----------------------------------------------------------------------------------------
John is in the hallway . Sandra is in the kitchen . Sandra journeyed to the bedroom . Mary travelled to the garden . Mary went back to the bathroom . Sandra is no longer in the bedroom . Is Sandra in the bedroom ? | Prediction: no | Ground Truth: no
-----------------------------------------------------------------------------------------
John is in the hallway . Sandra is in the kitchen . Sandra journeyed to the bedroom . Mary travelled to the garden . Mary went back to the bathroom . Sandra is no longer in the bedroo

In [27]:
print(story)

['Sandra', 'is', 'no', 'longer', 'in', 'the', 'bedroom', '.', 'John', 'went', 'back', 'to', 'the', 'kitchen', '.', 'John', 'is', 'in', 'the', 'hallway', '.', 'Daniel', 'is', 'in', 'the', 'hallway', '.', 'Daniel', 'is', 'no', 'longer', 'in', 'the', 'hallway', '.', 'John', 'moved', 'to', 'the', 'office', '.', 'John', 'travelled', 'to', 'the', 'bathroom', '.', 'Mary', 'is', 'in', 'the', 'bedroom', '.', 'Sandra', 'is', 'in', 'the', 'office', '.', 'Daniel', 'journeyed', 'to', 'the', 'garden', '.']


In [28]:
from keras.models import model_from_json
model_json = model.to_json()
with open("model_simple_negation.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_simple_negation.h5")
print("Saved model to disk")

Saved model to disk


In [29]:
json_file = open('model_simple_negation.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_simple_negation.h5")

C:\Users\pruth\Anaconda3\lib\site-packages\keras\engine\topology.py:1242: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [30]:
test_stories[0]

(['John',
  'is',
  'in',
  'the',
  'hallway',
  '.',
  'Sandra',
  'is',
  'in',
  'the',
  'kitchen',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'bedroom', '?'],
 'no')

In [31]:
loaded_model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
for i in range(0,10):
        current_inp = test_stories[i]
        current_story, current_query, current_answer = vectorize_stories([current_inp], word_idx, story_maxlen, query_maxlen)
        current_prediction = loaded_model.predict([current_story, current_query])
        current_prediction = idx_word[np.argmax(current_prediction)]
        print(' '.join(current_inp[0]), ' '.join(current_inp[1]), '| Prediction:', current_prediction, '| Ground Truth:', current_inp[2])
        print("-----------------------------------------------------------------------------------------")


John is in the hallway . Sandra is in the kitchen . Is Sandra in the bedroom ? | Prediction: no | Ground Truth: no
-----------------------------------------------------------------------------------------
John is in the hallway . Sandra is in the kitchen . Sandra journeyed to the bedroom . Mary travelled to the garden . Is Sandra in the bedroom ? | Prediction: yes | Ground Truth: yes
-----------------------------------------------------------------------------------------
John is in the hallway . Sandra is in the kitchen . Sandra journeyed to the bedroom . Mary travelled to the garden . Mary went back to the bathroom . Sandra is no longer in the bedroom . Is Sandra in the bedroom ? | Prediction: no | Ground Truth: no
-----------------------------------------------------------------------------------------
John is in the hallway . Sandra is in the kitchen . Sandra journeyed to the bedroom . Mary travelled to the garden . Mary went back to the bathroom . Sandra is no longer in the bedroo

In [32]:
print(current_story)

[[ 0  0  0  0  0  0  0  6 15 23 22 21  1  6 25  8 23 22 11  1  3 14 19 17
  13 22 16  1  5 14 19 17 13 22  9  1  5 14 13 22 11  1  7 24 23 22 10  1
   6 24 23 22 16  1  7 14 20 13 22 10  1  5 25  8 23 22 12  1  3 24 23 22
   9  1]]


In [33]:
print(current_query)

[[ 4  3 13 22  9  2]]


In [34]:
print(len(current_query))

1


In [35]:
print(test_stories[0])

(['John', 'is', 'in', 'the', 'hallway', '.', 'Sandra', 'is', 'in', 'the', 'kitchen', '.'], ['Is', 'Sandra', 'in', 'the', 'bedroom', '?'], 'no')
